In [1]:
from bs4 import BeautifulSoup
import requests

#  설계

## 클래스

### WebtoonData
웹툰 하나에 해당

-- 기존보기 --
- 제목 (title)
- 커버이미지 (url_img_thumbnail)

-- 상세보기 --
- 작가 (author)
- 작품설명 (description)

### EpisodeData
웹툰의 한 회에 해당

- 제목 (title)
- 커버이미지 (url_img_thumbnail)
- 별점 (rating)
- 등록일 (created_date)

In [2]:
# URL 지정 후 html 가져오기

webtoon_weekday_url = "https://comic.naver.com/webtoon/weekday.nhn"
html = requests.get(webtoon_weekday_url)

In [3]:
# 파일 쓰기

with open('weekday.html', 'wt') as f:
    f.write(html.text)

In [4]:
# 파일 불러오기

with open('weekday.html') as f:
    read_file_html = f.read()

In [5]:
# BeautifulSoup로 변환

soup = BeautifulSoup(read_file_html, 'lxml')

In [6]:
# 웹툰 각각, li 태그 내부의 요소들을 가져옴

each_webtoons = soup.select('.list_area')[0].select('.col > .col_inner > ul > li')

In [7]:
# 총 220개의 요소가 존재

len(each_webtoons)

220

In [21]:
# WebtoonData 클래스 정의
# Description, Author 요소는 트래픽 사용량이 많아 로직만 맨 아래 정의

class WebtoonData:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        
    def __str__(self):
        return self.title
    
    def __repr__(self):
        return self.title
    
    def __eq__(self, other):
        return self.title == other

In [25]:
# weekday.html에서 Title, Url_Img_Thumbnail 요소 가져오기 및 중복 제거

webtoon_list = []

for webtoon in each_webtoons:
    title = webtoon.select('a[class="title"]')[0].get_text()
    url_img_thumbnail = webtoon.select('.thumb > a > img')[0]['src']
    
    if title in webtoon_list:
        continue
    
    webtoon_list.append(WebtoonData(title, url_img_thumbnail))
    
print('중복제거 전 : ', len(each_webtoons), '개')
print('중복제거 후 : ', len(webtoon_list), '개')

중복제거 전 :  220 개
중복제거 후 :  192 개


In [12]:
# Description, Author 가져오기
# 트래픽 사용에 제한을 주기위해 3개만 가져오도록 했습니다.

based_url = 'http://comic.naver.com'


count = 0

for webtoon in each_webtoons:
    on_urls = webtoon.select('.thumb > a')[0]['href']
    
    if count == 3:
        break

    count += 1

    url = based_url + str(on_urls)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    author = soup.select('.detail > h2 > .wrt_nm')[0].get_text().strip()
    description = soup.select('.detail > p')[0].get_text().strip()
    print('작가이름 : ', author)
    print('설명 : ', description)

작가이름 :  SIU
설명 :  자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑
작가이름 :  설이 / 윤성원
설명 :  '여자도 군대에 간다면?'본격 여자도 군대 가는 만화!본 작품은 2014 네이버 수퍼루키 공모전 당선작품입니다.
작가이름 :  조용석
설명 :  자유를 꿈꾸는 청춘들의 스트릿라이딩 드라마
